In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import ast

import os

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

In [57]:
df = pd.read_csv('/content/drive/My Drive/266_project/mistral_7b_data/entity_predictions.csv')
df2 = pd.read_csv('/content/drive/My Drive/266_project/mistral_7b_data/entity_predictions_100.csv')

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows' ,None)
pd.set_option('display.width', None)

In [ ]:
df.head()

In [ ]:
# def compare_predictions(row):
#     results = {
#         'correct': [],
#         'incorrect': []
#     }

#     if pd.notna(row['Prediction']):
#         prediction = ast.literal_eval(row['Prediction'])  # Safer than eval
#     else:
#         prediction = {}  # Use an empty dictionary if prediction is NaN

#     true_value = ast.literal_eval(row['True Value'])

#     # keys to compare - union of both dictionaries' keys
#     all_keys = set(prediction.keys()) | set(true_value.keys())

#     for key in all_keys:
#         pred_val = prediction.get(key, [])  # Defaults to an empty list if key is not in prediction
#         true_val = true_value.get(key, [])  # Defaults to an empty list if key is not in true_value

#         if pred_val == true_val and pred_val != []:  # Correct prediction
#             results['correct'].append(key)
#         elif pred_val != true_val:  # Incorrect prediction
#             results['incorrect'].append(key)

#     return results


# df['Comparison_Results'] = df.apply(compare_predictions, axis=1)

# df[['Comparison_Results']]


In [ ]:
df['True Value'] = df['True Value'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
df['Prediction'] = df['Prediction'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# fn for true pos, false pos, false neg
def calculate_metrics(row):
    true_entities = set(row['True Value'].get('Drug', []))
    predicted_entities = set(row['Prediction'].get('Drug', []))

    tp = len(true_entities & predicted_entities)  # Intersection gives True Positives
    fp = len(predicted_entities - true_entities)   # Difference gives False Positives
    fn = len(true_entities - predicted_entities)   # Difference gives False Negatives

    return pd.Series([fp, fn, tp], index=['False Positives', 'False Negatives', 'True Positives'])

df[['False Positives', 'False Negatives', 'True Positives']] = df.apply(calculate_metrics, axis=1)
df2[['False Positives', 'False Negatives', 'True Positives']] = df2.apply(calculate_metrics, axis=1)

In [ ]:

# fn to safely evaluate a string or return the input if it's already a dictionary
def safe_eval(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return x
    return x

df['True Value'] = df['True Value'].apply(safe_eval)
df['Prediction'] = df['Prediction'].apply(safe_eval)

# calculate_metrics function
def calculate_metrics(row):
    fp_list, fn_list, tp_list = [], [], []
    all_keys = set(row['True Value'].keys()) | set(row['Prediction'].keys())

    for key in all_keys:
        true_entities = set(row['True Value'].get(key, []))
        predicted_entities = set(row['Prediction'].get(key, []))

        tp = len(true_entities & predicted_entities)
        fp = len(predicted_entities - true_entities)
        fn = len(true_entities - predicted_entities)

        fp_list.append(fp)
        fn_list.append(fn)
        tp_list.append(tp)

    total_fp = sum(fp_list)
    total_fn = sum(fn_list)
    total_tp = sum(tp_list)

    return pd.Series([total_fp, total_fn, total_tp], index=['False Positives', 'False Negatives', 'True Positives'])

df[['False Positives', 'False Negatives', 'True Positives']] = df.apply(calculate_metrics, axis=1)
df2[['False Positives', 'False Negatives', 'True Positives']] = df2.apply(calculate_metrics, axis=1)

In [53]:
# Calculating precision, recall, and F1 score
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Handling potential division by zero
df.fillna(0, inplace=True)

# Calculate the averages for Precision, Recall, and F1 Score
average_precision = df['Precision'].mean()
average_recall = df['Recall'].mean()
average_f1_score = df['F1 Score'].mean()

average_precision, average_recall, average_f1_score

(0.3353333333333333, 0.337, 0.33099999999999996)

In [82]:
df_filtered = df2[df2['Prediction'].apply(lambda x: bool(x))]

In [85]:
df2 =df_filtered.reset_index()

In [86]:
df2['Precision'] = df2['True Positives'] / (df2['True Positives'] + df2['False Positives'])
df2['Recall'] = df2['True Positives'] / (df2['True Positives'] + df2['False Negatives'])
df2['F1 Score'] = 2 * (df2['Precision'] * df2['Recall']) / (df2['Precision'] + df2['Recall'])

df2.fillna(0, inplace=True)

average_precision = df2['Precision'].mean()
average_recall = df2['Recall'].mean()
average_f1_score = df2['F1 Score'].mean()

average_precision, average_recall, average_f1_score

(0.6572139303482587, 0.6838308457711442, 0.659077986689927)